In [1]:
# Necessery library install
!pip install mysql-connector-python sqlalchemy pandas pymysql

In [6]:
# List all tables and their row counts
import pandas as pd
from sqlalchemy import create_engine, text

# Connection details
username = "root"
password = "kikodory"
host = "127.0.0.1"
port = 3306
database = "inventorydb"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

query = f"""
SELECT 
    table_name AS TableName,
    table_rows AS RowCount
FROM information_schema.tables
WHERE table_schema = '{database}'
ORDER BY table_name;
"""

with engine.connect() as conn:
    tables_df = pd.read_sql(text(query), conn)

print("📋 Tables in database and their row counts:")
display(tables_df)


📋 Tables in database and their row counts:


,TableName,RowCount
0,begin_inventory,199921
1,end_inventory,223912
2,purchase_prices,12271
3,purchase_summary,10812
4,purchases,2277911
5,sales,1047431
6,vendor_invoice,5484
7,vendor_sales_summary,0


In [7]:
# Query -> cast numeric fields in SQL, return DataFrame named vendor_sales_summary
import time
import pandas as pd
from sqlalchemy import text

query = """
WITH FreightSummary AS (
    SELECT
        CAST(VendorNumber AS SIGNED) AS VendorNumber,
        SUM(CAST(REPLACE(REPLACE(REPLACE(Freight, ',', ''), '$', ''), ' ', '') AS DECIMAL(18,2))) AS FreightCost
    FROM vendor_invoice
    GROUP BY CAST(VendorNumber AS SIGNED)
),
PurchaseSummary AS (
    SELECT
        CAST(p.VendorNumber AS SIGNED)                       AS VendorNumber,
        p.VendorName,
        CAST(p.Brand AS SIGNED)                              AS Brand,
        p.Description,
        CAST(p.PurchasePrice AS DECIMAL(14,2))               AS PurchasePrice,
        CAST(pp.Price AS DECIMAL(14,2))                      AS ActualPrice,
        CAST(pp.Volume AS DECIMAL(14,2))                     AS Volume,
        SUM(CAST(REPLACE(REPLACE(REPLACE(p.Quantity, ',', ''), '$', ''), ' ', '') AS DECIMAL(18,2))) AS TotalPurchaseQuantity,
        SUM(CAST(REPLACE(REPLACE(REPLACE(p.Dollars,  ',', ''), '$', ''), ' ', '') AS DECIMAL(18,2))) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
    WHERE CAST(p.PurchasePrice AS DECIMAL(14,2)) > 0
    GROUP BY CAST(p.VendorNumber AS SIGNED), p.VendorName, CAST(p.Brand AS SIGNED), p.Description, CAST(p.PurchasePrice AS DECIMAL(14,2)), CAST(pp.Price AS DECIMAL(14,2)), CAST(pp.Volume AS DECIMAL(14,2))
),
SalesSummary AS (
    SELECT
        CAST(VendorNumber AS SIGNED) AS VendorNumber,
        CAST(Brand AS SIGNED)        AS Brand,
        SUM(CAST(REPLACE(REPLACE(REPLACE(SalesQuantity, ',', ''), '$', ''), ' ', '') AS DECIMAL(18,2))) AS TotalSalesQuantity,
        SUM(CAST(REPLACE(REPLACE(REPLACE(SalesDollars,  ',', ''), '$', ''), ' ', '') AS DECIMAL(18,2))) AS TotalSalesDollars,
        SUM(CAST(REPLACE(REPLACE(REPLACE(SalesPrice,    ',', ''), '$', ''), ' ', '') AS DECIMAL(18,2))) AS TotalSalesPrice,
        SUM(CAST(REPLACE(REPLACE(REPLACE(ExciseTax,     ',', ''), '$', ''), ' ', '') AS DECIMAL(18,2))) AS TotalExciseTax
    FROM sales
    GROUP BY CAST(VendorNumber AS SIGNED), CAST(Brand AS SIGNED)
)
SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    COALESCE(ss.TotalSalesQuantity, 0) AS TotalSalesQuantity,
    COALESCE(ss.TotalSalesDollars, 0)  AS TotalSalesDollars,
    COALESCE(ss.TotalSalesPrice, 0)    AS TotalSalesPrice,
    COALESCE(ss.TotalExciseTax, 0)     AS TotalExciseTax,
    COALESCE(fs.FreightCost, 0)        AS FreightCost
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNumber
   AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC;
"""

# execute safely using SQLAlchemy 2.x + pandas
start = time.time()
with engine.connect() as conn:
    it = pd.read_sql(text(query), conn, chunksize=20000)   # reduce chunksize if memory/timeout occurs
    vendor_sales_summary = pd.concat(it, ignore_index=True)
end = time.time()

print("Query execution time (s):", round(end - start, 2))
print("Rows returned:", len(vendor_sales_summary))
display(vendor_sales_summary.head())

# ensure pandas dtypes are numeric where expected (extra safety)
to_numeric_cols = ["VendorNumber", "Brand", "PurchasePrice", "ActualPrice", "Volume"]
for c in to_numeric_cols:
    vendor_sales_summary[c] = pd.to_numeric(vendor_sales_summary[c], errors="coerce")

print("\nFinal dtypes:")
print(vendor_sales_summary.dtypes)


Query execution time (s): 73.46
Rows returned: 10692


,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750.0,145080.0,3811251.60,9578.0,344712.22,64889.97,17598.14,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750.0,164038.0,3804041.22,9203.0,275162.97,52289.50,16909.12,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750.0,187407.0,3418303.68,11189.0,288135.11,48202.30,20557.97,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750.0,201682.0,3261197.94,20226.0,444810.74,43304.31,37163.76,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750.0,138109.0,3023206.01,11883.0,357759.17,52774.51,21833.58,257032.07



Final dtypes:
VendorNumber               int64
VendorName                object
Brand                      int64
Description               object
PurchasePrice            float64
ActualPrice              float64
Volume                   float64
TotalPurchaseQuantity    float64
TotalPurchaseDollars     float64
TotalSalesQuantity       float64
TotalSalesDollars        float64
TotalSalesPrice          float64
TotalExciseTax           float64
FreightCost              float64
dtype: object


In [8]:
vendor_sales_summary

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750.0,145080.0,3811251.60,9578.0,344712.22,64889.97,17598.14,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750.0,164038.0,3804041.22,9203.0,275162.97,52289.50,16909.12,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750.0,187407.0,3418303.68,11189.0,288135.11,48202.30,20557.97,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750.0,201682.0,3261197.94,20226.0,444810.74,43304.31,37163.76,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750.0,138109.0,3023206.01,11883.0,357759.17,52774.51,21833.58,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750.0,2.0,2.64,3.0,5.97,5.97,0.33,27100.41
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50.0,6.0,2.34,128.0,62.72,0.98,6.72,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50.0,2.0,1.48,0.0,0.00,0.00,0.00,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200.0,1.0,1.47,6.0,11.94,7.96,1.26,257032.07


In [9]:
#change data type
import pandas as pd

# Change data types in vendor_sales_summary
vendor_sales_summary['TotalPurchaseQuantity'] = (
    pd.to_numeric(vendor_sales_summary['TotalPurchaseQuantity'], errors='coerce')
    .fillna(0)
    .round()
    .astype('int64')
)

float_cols = [
    'PurchasePrice','ActualPrice','TotalPurchaseDollars','TotalSalesQuantity',
    'TotalSalesDollars','TotalSalesPrice','TotalExciseTax','FreightCost','Volume'
]
for c in float_cols:
    if c in vendor_sales_summary.columns:
        vendor_sales_summary[c] = pd.to_numeric(vendor_sales_summary[c], errors='coerce').astype('float64')

for c in ['VendorNumber','Brand']:
    if c in vendor_sales_summary.columns:
        vendor_sales_summary[c] = pd.to_numeric(vendor_sales_summary[c], errors='coerce').fillna(0).astype('int64')

print("Final dtypes:")
print(vendor_sales_summary.dtypes)


Final dtypes:
VendorNumber               int64
VendorName                object
Brand                      int64
Description               object
PurchasePrice            float64
ActualPrice              float64
Volume                   float64
TotalPurchaseQuantity      int64
TotalPurchaseDollars     float64
TotalSalesQuantity       float64
TotalSalesDollars        float64
TotalSalesPrice          float64
TotalExciseTax           float64
FreightCost              float64
dtype: object


In [10]:
#check for null
vendor_sales_summary.isnull().sum()

VendorNumber             0
VendorName               0
Brand                    0
Description              0
PurchasePrice            0
ActualPrice              0
Volume                   0
TotalPurchaseQuantity    0
TotalPurchaseDollars     0
TotalSalesQuantity       0
TotalSalesDollars        0
TotalSalesPrice          0
TotalExciseTax           0
FreightCost              0
dtype: int64

In [11]:
#check for empty spaces
vendor_sales_summary['VendorName'].unique()

array(['BROWN-FORMAN CORP          ', 'MARTIGNETTI COMPANIES',
       'PERNOD RICARD USA          ', 'DIAGEO NORTH AMERICA INC   ',
       'BACARDI USA INC            ', 'JIM BEAM BRANDS COMPANY    ',
       'MAJESTIC FINE WINES        ', 'ULTRA BEVERAGE COMPANY LLP ',
       'STOLI GROUP,(USA) LLC      ', 'PROXIMO SPIRITS INC.       ',
       'MOET HENNESSY USA INC      ', 'CAMPARI AMERICA            ',
       'SAZERAC CO INC             ', 'CONSTELLATION BRANDS INC   ',
       'M S WALKER INC             ', 'SAZERAC NORTH AMERICA INC. ',
       'PALM BAY INTERNATIONAL INC ', 'REMY COINTREAU USA INC     ',
       'SIDNEY FRANK IMPORTING CO  ', 'E & J GALLO WINERY         ',
       'WILLIAM GRANT & SONS INC   ', 'HEAVEN HILL DISTILLERIES   ',
       'DISARONNO INTERNATIONAL LLC', 'EDRINGTON AMERICAS         ',
       'CASTLE BRANDS CORP.        ', 'SOUTHERN WINE & SPIRITS NE ',
       'STE MICHELLE WINE ESTATES  ', 'TRINCHERO FAMILY ESTATES   ',
       'MHW LTD                    ', 'W

In [12]:
vendor_sales_summary['Description'].unique()

array(['Jack Daniels No 7 Black', "Tito's Handmade Vodka",
       'Absolut 80 Proof', ..., 'Crown Royal Apple',
       'Concannon Glen Ellen Wh Zin', 'The Club Strawbry Margarita'],
      dtype=object)

In [13]:
#remove unnecessery spaces
vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()

In [14]:
# Create new analysis columns

# Gross Profit (Sales - Purchase)
vendor_sales_summary['GrossProfit'] = (
    vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']
)

# Profit Margin (%), safe divide
vendor_sales_summary['ProfitMargin'] = (
    vendor_sales_summary['GrossProfit'] / vendor_sales_summary['TotalSalesDollars']
).replace([float('inf'), -float('inf')], 0).fillna(0) * 100

# Stock Turnover (Sales Quantity / Purchase Quantity), safe divide
vendor_sales_summary['StockTurnover'] = (
    vendor_sales_summary['TotalSalesQuantity'] / vendor_sales_summary['TotalPurchaseQuantity']
).replace([float('inf'), -float('inf')], 0).fillna(0)

# Sales-to-Purchase Ratio (Dollars)
vendor_sales_summary['SalestoPurchaseRatio'] = (
    vendor_sales_summary['TotalSalesDollars'] / vendor_sales_summary['TotalPurchaseDollars']
).replace([float('inf'), -float('inf')], 0).fillna(0)

print("Added new analysis columns ✅")
display(vendor_sales_summary.head())


Added new analysis columns ✅


,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost,GrossProfit,ProfitMargin,StockTurnover,SalestoPurchaseRatio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750.0,145080,3811251.60,9578.0,344712.22,64889.97,17598.14,68601.68,-3466539.38,-1005.632867,0.066019,0.090446
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750.0,164038,3804041.22,9203.0,275162.97,52289.50,16909.12,144929.24,-3528878.25,-1282.468440,0.056103,0.072334
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750.0,187407,3418303.68,11189.0,288135.11,48202.30,20557.97,123780.22,-3130168.57,-1086.354443,0.059704,0.084292
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750.0,201682,3261197.94,20226.0,444810.74,43304.31,37163.76,257032.07,-2816387.20,-633.165287,0.100287,0.136395
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750.0,138109,3023206.01,11883.0,357759.17,52774.51,21833.58,257032.07,-2665446.84,-745.039419,0.086041,0.118338


In [15]:
vendor_sales_summary

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost,GrossProfit,ProfitMargin,StockTurnover,SalestoPurchaseRatio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750.0,145080,3811251.60,9578.0,344712.22,64889.97,17598.14,68601.68,-3466539.38,-1005.632867,0.066019,0.090446
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750.0,164038,3804041.22,9203.0,275162.97,52289.50,16909.12,144929.24,-3528878.25,-1282.468440,0.056103,0.072334
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750.0,187407,3418303.68,11189.0,288135.11,48202.30,20557.97,123780.22,-3130168.57,-1086.354443,0.059704,0.084292
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750.0,201682,3261197.94,20226.0,444810.74,43304.31,37163.76,257032.07,-2816387.20,-633.165287,0.100287,0.136395
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750.0,138109,3023206.01,11883.0,357759.17,52774.51,21833.58,257032.07,-2665446.84,-745.039419,0.086041,0.118338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750.0,2,2.64,3.0,5.97,5.97,0.33,27100.41,3.33,55.778894,1.500000,2.261364
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50.0,6,2.34,128.0,62.72,0.98,6.72,50293.62,60.38,96.269133,21.333333,26.803419
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50.0,2,1.48,0.0,0.00,0.00,0.00,14069.87,-1.48,0.000000,0.000000,0.000000
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200.0,1,1.47,6.0,11.94,7.96,1.26,257032.07,10.47,87.688442,6.000000,8.122449


In [17]:
import pandas as pd
from sqlalchemy import create_engine, text

# 1. Database connection info
username = "root"
password = "kikodory"
host = "127.0.0.1"
port = 3306
database = "inventorydb"

# 2. Create engine
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

# 3. Table creation SQL
create_table_sql = """
CREATE TABLE IF NOT EXISTS vendor_sales_summary (
    VendorNumber INT,
    VendorName VARCHAR(100),
    Brand INT,
    Description VARCHAR(255),
    PurchasePrice DECIMAL(10,2),
    ActualPrice DECIMAL(10,2),
    Volume INT,
    TotalPurchaseQuantity INT,
    TotalPurchaseDollars DECIMAL(15,2),
    TotalSalesQuantity INT,
    TotalSalesDollars DECIMAL(15,2),
    TotalSalesPrice DECIMAL(15,2),
    TotalExciseTax DECIMAL(15,2),
    FreightCost DECIMAL(15,2),
    GrossProfit DECIMAL(15,2),
    ProfitMargin DECIMAL(15,2),
    StockTurnover DECIMAL(15,2),
    SalesToPurchaseRatio DECIMAL(15,2),
    PRIMARY KEY (VendorNumber, Brand)
);
"""

# 4. Execute table creation
with engine.connect() as conn:
    conn.execute(text(create_table_sql))

print("✅ Table vendor_sales_summary created (if it did not already exist).")


✅ Table vendor_sales_summary created (if it did not already exist).


In [18]:
# Make sure vendor_sales_summary DataFrame exists
print(vendor_sales_summary.head())

# Push data into MySQL
vendor_sales_summary.to_sql(
    name="vendor_sales_summary",
    con=engine,
    if_exists="append",   # use 'replace' if you want to drop & recreate
    index=False           # don't write DataFrame index as a column
)

print("✅ Data inserted into vendor_sales_summary table.")


   VendorNumber                VendorName  Brand              Description  \
0          1128         BROWN-FORMAN CORP   1233  Jack Daniels No 7 Black   
1          4425     MARTIGNETTI COMPANIES   3405    Tito's Handmade Vodka   
2         17035         PERNOD RICARD USA   8068         Absolut 80 Proof   
3          3960  DIAGEO NORTH AMERICA INC   4261   Capt Morgan Spiced Rum   
4          3960  DIAGEO NORTH AMERICA INC   3545          Ketel One Vodka   

   PurchasePrice  ActualPrice  Volume  TotalPurchaseQuantity  \
0          26.27        36.99  1750.0                 145080   
1          23.19        28.99  1750.0                 164038   
2          18.24        24.99  1750.0                 187407   
3          16.17        22.99  1750.0                 201682   
4          21.89        29.99  1750.0                 138109   

   TotalPurchaseDollars  TotalSalesQuantity  TotalSalesDollars  \
0            3811251.60              9578.0          344712.22   
1            3804041